# BAB 1b, Data Collecting: Total Charts Scrapping
Sumber data: https://kworb.net/spotify/country/id_daily_totals.html

Hari / Tanggal: Selasa / 20 Juni 2023

In [ ]:
!pip install spotipy

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd
import datetime
import calendar

In [ ]:
def get_today_date():
    # Mendapatkan tanggal hari ini
    today = datetime.date.today()

    # Mendapatkan nama bulan dalam bahasa Indonesia
    month_name = calendar.month_name[today.month]

    # Mengubah format tanggal menjadi string dalam bentuk "DD MonthName YYYY"
    formatted_date = f"{today.day} {month_name} {today.year}"

    return formatted_date

In [ ]:
def create_spotify_playlist(dataframe, playlist_name, client_id, client_secret):
    # Menginisialisasi koneksi dengan Spotify API menggunakan Spotipy
    sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=client_id, client_secret=client_secret, redirect_uri='https://example.com/callback', scope='playlist-modify-private'))

    # Membuat playlist baru
    playlist = sp.user_playlist_create(sp.current_user()['id'], playlist_name, public=False)

    # Menambahkan lagu ke playlist berdasarkan kolom 'Artist' dan 'Title' dalam DataFrame
    for index, row in dataframe.iterrows():
        artist = row['Artist']
        title = row['Title']

        # Mencari lagu berdasarkan artist dan title
        results = sp.search(q=f'artist:{artist} track:{title}', type='track')

        # Memastikan ada hasil pencarian lagu
        if results['tracks']['total'] > 0:
            track_uri = results['tracks']['items'][0]['uri']

            # Menambahkan lagu ke playlist
            sp.playlist_add_items(playlist['id'], [track_uri])

    # Mengembalikan URL playlist yang dibuat
    return playlist['external_urls']['spotify']

In [ ]:
today_date = get_today_date()
playlist_name = f"Lagu Teratas Indonesia 2016-2023 - {today_date}"
client_id = '3ff80dfa932f42ccba937689112ebaf8'
client_secret = 'b222a7c17fda476aa56057b10f263a7d'

In [ ]:
# URL untuk mengakses tabel Spotify
url = "https://kworb.net/spotify/country/id_daily_totals.html"

# Membaca tabel dari URL menggunakan pandas
data = pd.read_html(url)

# Mengambil tabel pertama dari hasil pembacaan (indeks 0)
df = data[0]

# Mengganti nama kolom 'Artist and Title' menjadi 'Artist'
df.rename(columns={'Artist and Title': 'Artist'}, inplace=True)

# Membagi kolom 'Artist' menjadi 'Artist' dan 'Title'
df[['Artist', 'Title']] = df['Artist'].str.split(' - ', 1, expand=True)

<ipython-input-6-8114ddce8bad>:14: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  df[['Artist', 'Title']] = df['Artist'].str.split(' - ', 1, expand=True)


In [ ]:
df.shape

(4722, 8)

In [ ]:
df.head()

,Artist,Days,T10,Pk,(x?),PkStreams,Total,Title
0,Pamungkas,1274,238.0,1,(x80),750272,208336600,To the Bone
1,Feby Putri,617,434.0,1,(x4),596645,205823463,Runtuh
2,Tulus,473,260.0,1,(x89),1461480,201124998,Hati-Hati di Jalan
3,Nadin Amizah,1116,169.0,1,(x40),380554,191139969,Bertaut
4,Tulus,2378,NaN,12,NaN,421746,182870661,Monokrom


In [ ]:
from IPython.display import display, HTML

# URL untuk mendapatkan authorization code
auth_url = "https://accounts.spotify.com/authorize?client_id=CLIENT_ID&response_type=code&redirect_uri=REDIRECT_URI&scope=playlist-modify-private"

# Menampilkan URL sebagai tautan yang dapat diklik
display(HTML(f"Click <a href='{auth_url}' target='_blank'>here</a> to authorize access to your Spotify account."))

# Meminta pengguna untuk memasukkan authorization code secara manual
authorization_code = input("Enter the authorization code: ")

In [ ]:
# Membuat playlist di Spotify
playlist_url = create_spotify_playlist(df, playlist_name, client_id, client_secret)
print(f"Playlist '{playlist_name}' berhasil dibuat di Spotify. URL playlist: {playlist_url}")

Enter the URL you were redirected to: https://example.com/callback?code=AQBc9g4A6kStW9pKqe_rbP7xNX1OnIAAHqhKNdnBdQnGiTMqLgvnE7LnB0-PSGmxccl3GD2LWC2x-2fCop0ytwdR1NRaK-CQyKZHXTHM_sYCk7WOnbwSookPzIzAfeT1TZ0VOsxyRNokqQZgHEgjzF8kl6Ni7VZ-aR7G8qrIjM4t5DxDy09QZo7_gTkSzsJ_hcWOdFL1l7Pk
Playlist 'Lagu Teratas Indonesia 2016-2023 - 20 June 2023' berhasil dibuat di Spotify. URL playlist: https://open.spotify.com/playlist/5v4EtazJ8p90GXyFRiIwHT


In [ ]:
df.to_csv('df_total.csv')